In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime
import os

now = datetime.datetime.now().strftime("%Y%m%d")

In [2]:
def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

In [3]:
#new chrome driver
def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    option = webdriver.ChromeOptions()

    #fix chrome 103 problem
    option.binary_location = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

    prefs={"profile.default_content_setting_values.notifications" : 2}
    option.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=option)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [4]:
#抓取幾頁
def getPageNum(url,driver):
    driver.get(url)
            
    #Get Comment       
    soup = BeautifulSoup(driver.page_source,"html.parser")
    pages = soup.find("div", class_="article_multi_page clear")
    try:
        page_num = len(pages.find_all("a"))
    except:
        return 1
    else:
        return page_num 

In [5]:
#抓取貼文連結
def getPostLink(url,driver):
    driver.get(url)
    
    try:
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)
    except OSError:
        time.sleep(15)
        
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)   
    post_link_list = []
    soup = BeautifulSoup(driver.page_source,"html.parser")
    for l in soup.find_all("div", class_="content_col"):
        temp_soup = BeautifulSoup(str(l),"html.parser")
        temp_target = temp_soup.find("a")
        temp = []
        temp.append(temp_target.get_text().strip().split('\t'))
        temp.append("https://www.dcfever.com"+temp_target["href"])
        post_link_list.append(temp)
    link_df = pd.DataFrame(data=post_link_list, columns=['Post Title', 'Link'])
    return link_df

In [6]:
#取得貼文日期

def getPostDate(page_soup):
    date_tag_string = page_soup.find("div", class_="article_date").get_text()
    date_tag_string = date_tag_string[date_tag_string.find("發表於")+3:date_tag_string.find("日")]
    year = date_tag_string[:date_tag_string.find("年")].strip()
    month = date_tag_string[date_tag_string.find("年")+1:date_tag_string.find("月")].strip()
    day = date_tag_string[date_tag_string.find("月")+1:].strip()
    post_date = datetime.date(int(year),int(month),int(day))
    return post_date.strftime("%Y/%m/%d")


In [7]:
#抓取貼文留言
def getPostContent(first_url,driver):
    
    urls=[]
    content_all=[]
    post_date = ""
    
    page_num = getPageNum(first_url,driver)
    if (page_num>1):
        for i in range(1,page_num+1):
            urls.append(first_url+"&page="+str(i))
        for url in urls:
            driver.get(url)
        #Get Comment       
            soup = BeautifulSoup(driver.page_source,"html.parser")
            if (url.find("&page=1") != -1):
                post_date = getPostDate(soup)
        
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)
    
    else:
        driver.get(first_url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        post_date = getPostDate(soup)
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)

    FinishedCrawler(driver)
    return listToString(content_all), post_date

#option, adding getdata

phone_df = pd.read_csv("../crawler_data/Phone_Data_20220809.csv")
get_data_phone_df = pd.DataFrame(data=[], columns=['Brand_Name', 'Phone_Name', 'Link', 'Icon'])
path = "../crawler_data/"
dirs = os.listdir( path )
for brand_dir in dirs:
    if(brand_dir != ".DS_Store" and brand_dir != "較舊" and brand_dir.find("csv")==-1):
        temp_brand_dir = os.listdir( path+brand_dir+"/Link" )
        for phone in temp_brand_dir:
            temp_phone_row = phone_df.loc[phone_df['Phone_Name'] == phone]
            get_data_phone_df = pd.concat([get_data_phone_df, temp_phone_row], axis=0)

file_name = "../crawler_data/Get_Phone_Data_"+now+".csv"
#get_data_phone_df.to_csv(file_name, index=False, encoding="utf-8")

get_data_phone_df = pd.read_csv("../crawler_data/Get_Phone_Data_20220816.csv")

dcfever_posts_df = pd.DataFrame(data=[], columns=['Post Title', 'Link'])
phone_name = []
for index, row in get_data_phone_df.iterrows():
    try:
        driver = OpenChromeDriver()
        report_link = row["Link"].replace("specification","articles")
        temp_phone_link_df = getPostLink(report_link, driver)
        dcfever_posts_df = pd.concat([dcfever_posts_df, temp_phone_link_df], axis=0)
        for i in range(len(temp_phone_link_df)):
            phone_name.append(row["Phone_Name"])
        time.sleep(0.5)
    except:
        print(row["Phone_Name"])
    finally:
        FinishedCrawler(driver)

dcfever_posts_df["Phone_Name"] = phone_name
dcfever_posts_df


file_name = "../crawler_data/Temp_DcFever_Phone_Data_"+now+".csv"
dcfever_posts_df.to_csv(file_name, index=False, encoding="utf-8")

In [8]:
dcfever_posts_df = pd.read_csv("../crawler_data/Temp_DcFever_Phone_Data_20220816.csv")
dcfever_posts_df

,Post Title,Link,Phone_Name
0,['iPhone SE 3 詳測：大量引誘吸 Android 客'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
1,['iPhone SE 3 開箱：配件有一些轉變'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
2,['iPhone SE 3 親民價發表：5G、A15 SoC 外還有不少強化'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3
3,['【iOS 15.4 新功能】iPhone 13 Pro 戴口罩使用 Face ID 實試'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13
4,['iPhone iOS 15.1 更新：新增 ProRes 影片及 FaceTime 同播...,https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13
...,...,...,...
172,['【攝力分析】Sony Xperia Pro-I 終於有機取代一般 DC？'],https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
173,"['Sony Xperia Pro-I 實試：1"" Sensor 手機威力先睹為快']",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
174,"['Sony Xperia Pro-I 發表香港有價：不一樣的 1"" Sensor、首發 4...",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I
175,['vivo V23 5G 測試：5000 萬像素前鏡自拍多玩法'],https://www.dcfever.com/news/readnews.php?id=3...,vivo V23 5G


path = "../crawler_data/"
dirs = os.listdir( path )
for brand_dir in dirs:
    if(brand_dir != ".DS_Store" and brand_dir != "較舊" and brand_dir.find("csv")==-1):
        temp_path = path+brand_dir+"/Link"
        temp_brand_dir = os.listdir(temp_path)
        for phone in temp_brand_dir:
            temp_file_name = temp_path + "/" + phone + "/DCfever_" + phone.replace(" ","") +"_Link"+now+".csv"
            temp_phone_df = dcfever_posts_df.loc[dcfever_posts_df['Phone_Name'] == phone]
            if(len(temp_phone_df)>0):
                temp_phone_df.to_csv(temp_file_name, index=False, encoding="utf-8")
            else:
                continue

In [9]:
phone_data_df = pd.read_csv("../crawler_data/Phone_Data_20220809.csv")
phone_data_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,iPhone SE 3,https://www.dcfever.com/phones/specification.p...,../icon/iPhone SE 3.jpg
1,apple,iPhone 13 mini,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 mini.jpg
2,apple,iPhone 13,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13.jpg
3,apple,iPhone 13 Pro,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro.jpg
4,apple,iPhone 13 Pro Max,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro Max.jpg
...,...,...,...,...
1088,sharp,Sharp SX633A,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SX633A.jpg
1089,sharp,Sharp 102SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 102SH.jpg
1090,sharp,Sharp 104SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 104SH.jpg
1091,sharp,Sharp 816SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 816SH.jpg


In [10]:
#read data fixing
brand = []
for index, row in dcfever_posts_df.iterrows():
    temp_brand_name = phone_data_df.loc[phone_data_df['Phone_Name'] == row['Phone_Name']]['Brand_Name'].values[0]
    brand.append(temp_brand_name)
dcfever_posts_df['Brand_Name'] = brand
dcfever_posts_df

,Post Title,Link,Phone_Name,Brand_Name
0,['iPhone SE 3 詳測：大量引誘吸 Android 客'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3,apple
1,['iPhone SE 3 開箱：配件有一些轉變'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3,apple
2,['iPhone SE 3 親民價發表：5G、A15 SoC 外還有不少強化'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone SE 3,apple
3,['【iOS 15.4 新功能】iPhone 13 Pro 戴口罩使用 Face ID 實試'],https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13,apple
4,['iPhone iOS 15.1 更新：新增 ProRes 影片及 FaceTime 同播...,https://www.dcfever.com/news/readnews.php?id=3...,iPhone 13,apple
...,...,...,...,...
172,['【攝力分析】Sony Xperia Pro-I 終於有機取代一般 DC？'],https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I,sony
173,"['Sony Xperia Pro-I 實試：1"" Sensor 手機威力先睹為快']",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I,sony
174,"['Sony Xperia Pro-I 發表香港有價：不一樣的 1"" Sensor、首發 4...",https://www.dcfever.com/news/readnews.php?id=3...,Sony Xperia Pro-I,sony
175,['vivo V23 5G 測試：5000 萬像素前鏡自拍多玩法'],https://www.dcfever.com/news/readnews.php?id=3...,vivo V23 5G,vivo


path = "../crawler_data/"
temp_phone_name = ""
contents = []
dates = []
paths = []
for index, row in dcfever_posts_df.iterrows():
    temp_phone = row['Phone_Name']
    link = row['Link']
    temp_path = path + row['Brand_Name'] + '/Data/' + temp_phone + "/DCfever_" + temp_phone.replace(' ', '') + "_Data" + now + ".csv"
    try:
        temp_content, temp_date = getPostContent(link,OpenChromeDriver())
        contents.append(temp_content)
        dates.append(temp_date)
        paths.append(temp_path)
    except:
        print(temp_phone_name)
    finally:
        time.sleep(0.5)


path = "../crawler_data/"
dirs = os.listdir( path )
for brand_dir in dirs:
    if(brand_dir != ".DS_Store" and brand_dir != "較舊" and brand_dir.find("csv")==-1):
        temp_path = path+brand_dir+"/Link"
        temp_data_path = path+brand_dir+"/Data"
        temp_brand_dir = os.listdir(temp_path)
        for phone in temp_brand_dir:
            temp_phone_link_path = temp_path + "/"+phone
            temp_phone_links = os.listdir(temp_phone_link_path)
            for link in temp_phone_links:
                try:
                    if(link.find("DCfever")==0):
                        dcfever_link_path = temp_phone_link_path + "/" +link
                        link_df = pd.read_csv(dcfever_link_path)
                        temp_file_name = temp_data_path + "/" + phone + "/DCfever_" + phone.replace(" ","") +"_Data"+now+".csv"
                        contents = []
                        dates = []
                        for index, row in link_df.iterrows():
                            temp_link = row['Link']
                            temp_content, temp_date = getPostContent(temp_link,OpenChromeDriver())
                            contents.append(temp_content)
                            dates.append(temp_date)
                        content_df = pd.DataFrame({'Content': contents,'Date': dates})
                        all_df = link_df.join(content_df)
                        all_df.to_csv(temp_file_name, index=False, encoding="utf-8")
                    else:
                        continue
                except:
                    print(link)

#dcfever csv fix
crawler_data_path = "../crawler_data/"
brand_list_dir = os.listdir(crawler_data_path)
for brand in brand_list_dir:
    if(brand != ".DS_Store" and brand != "較舊" and brand.find("csv")==-1):
        brand_data_path = crawler_data_path + brand +"/Data/"
        phone_list_dir = os.listdir(brand_data_path)
        for phone in phone_list_dir:
            phone_data_path = brand_data_path + phone +'/'
            phone_data_list_dir = os.listdir(phone_data_path)
            for data in phone_data_list_dir:
                temp_csv_path = phone_data_path + data
                if(temp_csv_path.find('DCfever')!=-1):
                    temp_df = pd.read_csv(temp_csv_path)
                    temp_df = temp_df.drop(['Phone_Name','Brand_Name'], axis=1)
                    temp_df.to_csv(temp_csv_path, index=False, encoding="utf-8")